<a href="https://colab.research.google.com/github/NdopnnoabasiJames/LinearAndPolynomialRegressionModels/blob/main/LogisticRegressionOnPlantGrowth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Imports & settings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import warnings
warnings.filterwarnings("ignore")
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)


In [2]:
# Load and view dataset
df = pd.read_csv('/content/drive/MyDrive/ML_Datasets/Copy of plant_growth_data.csv')
df.head()

,Soil_Type,Sunlight_Hours,Water_Frequency,Fertilizer_Type,Temperature,Humidity,Growth_Milestone
0,loam,5.192294,bi-weekly,chemical,31.719602,61.591861,0
1,sandy,4.033133,weekly,organic,28.919484,52.422276,1
2,loam,8.892769,bi-weekly,none,23.179059,44.660539,0
3,loam,8.241144,bi-weekly,none,18.465886,46.433227,0
4,sandy,8.374043,bi-weekly,organic,18.128741,63.625923,0


In [3]:
# Step: identify features and target, inspect types and missing values

# 1) define target and feature set
target_col = "Growth_Milestone"
X = df.drop(columns=[target_col])
y = df[target_col]

# 2) quick checks
print("Target distribution:")
print(y.value_counts(dropna=False))
print("\nTarget dtype:", y.dtype)

print("\nFeature dtypes and sample values:")
print(X.dtypes)
print(X.head())

print("\nMissing values per column:")
print(X.isna().sum())

print("\nNumeric summary for numeric columns:")
print(X.select_dtypes(include=["number"]).describe().T)

print("\nUnique values for categorical columns (small sample):")
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
for c in cat_cols:
    print(f"\n{c}: {X[c].unique()[:10]} (showing up to 10 unique values)")

Target distribution:
Growth_Milestone
0    97
1    96
Name: count, dtype: int64

Target dtype: int64

Feature dtypes and sample values:
Soil_Type           object
Sunlight_Hours     float64
Water_Frequency     object
Fertilizer_Type     object
Temperature        float64
Humidity           float64
dtype: object
  Soil_Type  Sunlight_Hours Water_Frequency Fertilizer_Type  Temperature  \
0      loam        5.192294       bi-weekly        chemical    31.719602   
1     sandy        4.033133          weekly         organic    28.919484   
2      loam        8.892769       bi-weekly            none    23.179059   
3      loam        8.241144       bi-weekly            none    18.465886   
4     sandy        8.374043       bi-weekly         organic    18.128741   

    Humidity  
0  61.591861  
1  52.422276  
2  44.660539  
3  46.433227  
4  63.625923  

Missing values per column:
Soil_Type          0
Sunlight_Hours     0
Water_Frequency    0
Fertilizer_Type    0
Temperature        0
Humidity

**Quick summary of your findings**

	•	Target (Growth_Milestone) → numeric (0 or 1) →  means we’re doing a classification problem (predicting whether a plant reached a milestone or not).
	•	Numeric features → Sunlight_Hours, Temperature, Humidity
	•	Categorical features → Soil_Type, Water_Frequency, Fertilizer_Type
	•	No missing values →  nice and clean

**What we’ll do next**

Before we can train the model, we need to convert the categorical features (the text ones) into numeric form.

We’ll use One-Hot Encoding again because these are nominal categories (no order between them — “loam” isn’t greater or less than “sandy”).

# Step 2: Encode the categorical columns

In [4]:
# Identify categorical columns
cat_cols = ["Soil_Type", "Water_Frequency", "Fertilizer_Type"]

# Initialize encoder
encoder = OneHotEncoder(drop='first', sparse_output=False)  # drop='first' avoids dummy variable trap

# Fit and transform
encoded_array = encoder.fit_transform(X[cat_cols])

# Convert to DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(cat_cols))

# Merge encoded data with numeric columns
X_encoded = pd.concat([X.drop(columns=cat_cols).reset_index(drop=True), encoded_df], axis=1)

print("Encoded feature columns:\n", X_encoded.columns)
print("\nEncoded data sample:\n", X_encoded.head())

Encoded feature columns:
 Index(['Sunlight_Hours', 'Temperature', 'Humidity', 'Soil_Type_loam',
       'Soil_Type_sandy', 'Water_Frequency_daily', 'Water_Frequency_weekly',
       'Fertilizer_Type_none', 'Fertilizer_Type_organic'],
      dtype='object')

Encoded data sample:
    Sunlight_Hours  Temperature   Humidity  Soil_Type_loam  Soil_Type_sandy  \
0        5.192294    31.719602  61.591861             1.0              0.0   
1        4.033133    28.919484  52.422276             0.0              1.0   
2        8.892769    23.179059  44.660539             1.0              0.0   
3        8.241144    18.465886  46.433227             1.0              0.0   
4        8.374043    18.128741  63.625923             0.0              1.0   

   Water_Frequency_daily  Water_Frequency_weekly  Fertilizer_Type_none  \
0                    0.0                     0.0                   0.0   
1                    0.0                     1.0                   0.0   
2                    0.0        

Now all your features are numeric, meaning the model can understand them.

Let’s break down what we see before moving on:

1. You kept all your numeric columns (Sunlight_Hours, Temperature, Humidity).
2.	Each categorical column (like Soil_Type, Water_Frequency, Fertilizer_Type) was turned into dummy variables — columns of 0s and 1s showing the presence or absence of a category.
3.	You dropped the first category of each to avoid multicollinearity (the “dummy variable trap”), so if Soil_Type_clay was dropped, it’s represented when both Soil_Type_loam and Soil_Type_sandy are 0.

Everything looks clean and ready for modeling.

# Next Step: Split into Training and Testing Sets

Now we’ll divide our data so the model can learn on one portion and be evaluated on unseen data — same logic as with the fish dataset.


In [5]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

print("Training features shape:", X_train.shape)
print("Testing features shape:", X_test.shape)
print("Training target shape:", y_train.shape)
print("Testing target shape:", y_test.shape)

Training features shape: (154, 9)
Testing features shape: (39, 9)
Training target shape: (154,)
Testing target shape: (39,)


Here’s what that means in simple terms:

1. You have 154 samples for training (that’s 80% of your data).
2. You have 39 samples for testing (20% of your data).
3. Each sample has 9 features (your numeric + encoded columns).

So now your model can:
1. Learn from the 154 training samples
2. Prove what it learned on the 39 testing samples


next step is to train your logistic regression model and make predictions.

Let’s go ahead and do that now 👇🏽

In [6]:
from sklearn.linear_model import LogisticRegression

# Initialize and train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Compare actual vs predicted values
comparison = pd.DataFrame({
    "Actual": y_test.values,
    "Predicted": y_pred
})

print(comparison.head(10))

   Actual  Predicted
0       1          1
1       1          1
2       0          1
3       0          0
4       0          0
5       0          0
6       1          1
7       1          0
8       0          1
9       1          1


So you can already see your model is getting some right and some wrong, which is normal. No model is perfect.

### Next Step: Evaluate Your Model

We’ll calculate a few important metrics:

	1.	Accuracy → how often it’s right.
	2.	Confusion Matrix → shows exactly where it gets things wrong (like predicting 1 instead of 0).
	3.	Classification Report → gives extra details (precision, recall, F1-score).


In [7]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", round(accuracy, 3))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", cm)

# Classification Report
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.538

Confusion Matrix:
 [[ 9  8]
 [10 12]]

Classification Report:
               precision    recall  f1-score   support

           0       0.47      0.53      0.50        17
           1       0.60      0.55      0.57        22

    accuracy                           0.54        39
   macro avg       0.54      0.54      0.54        39
weighted avg       0.54      0.54      0.54        39



## Key Takeaways

1. Accuracy: 0.538 (≈ 54%)

This means your model got about 54% of all predictions correct.

So, out of 39 plants in your test data:

	•	Around 21 were correctly predicted,
	•	The rest (about 18) were wrong.

That’s just slightly better than random guessing (which would be 50% if we guessed Yes/No randomly).

It’s not bad for a first attempt — it just means there’s room for improvement (we’ll talk about how later).


2. Confusion Metrix

So:

	•	9 plants that were truly 0 (no milestone) were correctly predicted as 0 ✅
	•	8 plants that were 0 were wrongly predicted as 1 ❌
	•	12 plants that were truly 1 were correctly predicted as 1 ✅
	•	10 plants that were 1 were wrongly predicted as 0 ❌

💡 In short: the model sometimes mixes up the two classes — it’s predicting “Yes” for some that shouldn’t and “No” for some that should.

####  Classification Report

| Term | Meaning |
|------|----------|
| **Precision** | When the model predicts a certain class, how often is it right? |
| **Recall** | Out of all actual cases of that class, how many did the model detect correctly? |
| **F1-score** | A balance between precision and recall. The closer to 1, the better. |
| **Support** | Number of actual samples in each class. |

**Class 0 (No milestone):**  
- Precision: 0.47 → When the model predicts “No Milestone”, it’s correct about half the time.  
- Recall: 0.53 → It caught 53% of all the true “No Milestone” cases.  
- F1: 0.50 → Overall average performance for this class.

**Class 1 (Milestone achieved):**  
- Precision: 0.60 → When it predicts “Milestone achieved”, it’s right 60% of the time.  
- Recall: 0.55 → It correctly identified 55% of plants that truly reached their milestone.  
- F1: 0.57 → Slightly better performance than class 0.

**Overall:**  
Accuracy ≈ 54%  
The model is roughly balanced between both classes but not yet very reliable.

---

####  Why performance might be low
1. The available features may not strongly predict plant growth milestone.  
2. Categorical variables might need richer encoding or interaction terms.  
3. The numeric features aren’t scaled — logistic regression benefits from feature scaling.  
4. The model’s regularization strength (parameter `C`) could need tuning.

---

#### Takeaway
I’ve successfully built and evaluated a working logistic regression model.  
Now that you know where it struggles, the next step is **improving accuracy**  by scaling

## Feature Scaling (Standardization)

We’ll scale the numeric features only, because the encoded categorical features (0s and 1s) are already in the same small range.



### Feature Scaling

Logistic Regression uses gradient-based optimization, which works better when all features are in similar ranges.  

Scaling prevents features with larger values (like Temperature) from dominating those with smaller ones (like Sunlight_Hours).

We use **StandardScaler**, which converts values so that:

- Mean = 0
- Standard deviation = 1

This helps the model converge faster and perform more accurately.

In [8]:
from sklearn.preprocessing import StandardScaler

# Identify numeric columns only
num_cols = ['Sunlight_Hours', 'Temperature', 'Humidity']

# Initialize the scaler
scaler = StandardScaler()

# Fit only on training numeric data, then transform both train and test
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

# Preview scaled data
X_train.head()

,Sunlight_Hours,Temperature,Humidity,Soil_Type_loam,Soil_Type_sandy,Water_Frequency_daily,Water_Frequency_weekly,Fertilizer_Type_none,Fertilizer_Type_organic
82,0.108244,1.330352,0.072772,0.0,0.0,0.0,0.0,1.0,0.0
109,-1.232538,1.222067,0.823863,1.0,0.0,0.0,0.0,0.0,0.0
163,1.423160,0.231164,0.916617,1.0,0.0,0.0,1.0,0.0,1.0
35,-0.187927,0.444056,-1.860204,0.0,0.0,0.0,0.0,0.0,1.0
136,0.042871,1.241123,-0.807062,1.0,0.0,1.0,0.0,1.0,0.0


Now that everything’s on the same scale, your logistic regression model should be able to learn fairer relationships between features instead of being dominated by the larger ones.


### Retrain and Re-evaluate

We’ll now retrain the model on the scaled data and check if the performance improves.


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Retrain logistic regression on scaled data
model_scaled = LogisticRegression(max_iter=1000)
model_scaled.fit(X_train, y_train)

# Make predictions
y_pred_scaled = model_scaled.predict(X_test)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred_scaled)
cm = confusion_matrix(y_test, y_pred_scaled)
report = classification_report(y_test, y_pred_scaled)

print("Accuracy after scaling:", round(accuracy, 3))
print("\nConfusion Matrix:\n", cm)
print("\nClassification Report:\n", report)

Accuracy after scaling: 0.538

Confusion Matrix:
 [[ 9  8]
 [10 12]]

Classification Report:
               precision    recall  f1-score   support

           0       0.47      0.53      0.50        17
           1       0.60      0.55      0.57        22

    accuracy                           0.54        39
   macro avg       0.54      0.54      0.54        39
weighted avg       0.54      0.54      0.54        39



###Why scaling didn’t change the score

Scaling isn’t a magic accuracy booster — it doesn’t change your data relationships, it only helps the model learn more fairly.
If your model already converged fine before, scaling won’t make it smarter — it’ll just make it more stable internally.

So the model still predicts about 54% correctly, which means:

	•	It’s learning something, but
	•	It can’t clearly separate plants that reach milestones from those that don’t.

### Next Step: Tune the Model (Regularization Strength)

Now we’ll move to the next lever — regularization tuning.

Logistic regression includes a parameter C, which controls how much it penalizes complexity.

	•	Small C → stronger regularization (simpler model)
	•	Large C → weaker regularization (more flexible model)

Sometimes, the default (C=1.0) isn’t the best fit.

Let’s test a few different C values and see which performs best:

In [10]:
# Try different C values
C_values = [0.01, 0.1, 1, 10, 100]
accuracies = []

for c in C_values:
    model = LogisticRegression(C=c, max_iter=1000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)
    print(f"C={c}: Accuracy={round(acc, 3)}")

# Display which one performed best
best_C = C_values[np.argmax(accuracies)]
print("\nBest C value:", best_C)

C=0.01: Accuracy=0.462
C=0.1: Accuracy=0.615
C=1: Accuracy=0.538
C=10: Accuracy=0.538
C=100: Accuracy=0.538

Best C value: 0.1


Our model actually improved with C = 0.1, bumping accuracy from 0.538 → 0.615 (about 62%). That’s a clear sign the model was slightly overfitting before, and reducing the regularization strength helped it generalize better.

So your best-performing model is the moderately regularized one (C=0.1). That is Best balance between bias and variance


### What to Do Next

Now that we’ve found the best C, let’s:

	1.	Retrain the model with C=0.1
	2.	Check confusion matrix and classification report again to confirm improvement
	3.	Then we’ll decide if it’s time to move to a more advanced model (like Random Forest).


In [11]:
# Final tuned model
best_model = LogisticRegression(C=0.1, max_iter=1000)
best_model.fit(X_train, y_train)

# Predictions
y_pred_best = best_model.predict(X_test)

# Evaluation
acc = accuracy_score(y_test, y_pred_best)
cm = confusion_matrix(y_test, y_pred_best)
report = classification_report(y_test, y_pred_best)

print("Final Model Accuracy:", round(acc, 3))
print("\nConfusion Matrix:\n", cm)
print("\nClassification Report:\n", report)

Final Model Accuracy: 0.615

Confusion Matrix:
 [[11  6]
 [ 9 13]]

Classification Report:
               precision    recall  f1-score   support

           0       0.55      0.65      0.59        17
           1       0.68      0.59      0.63        22

    accuracy                           0.62        39
   macro avg       0.62      0.62      0.61        39
weighted avg       0.63      0.62      0.62        39



###  Final Model Evaluation (After Regularization Tuning)

**Accuracy:** `0.615`  
The model now correctly predicts about **61.5%** of all plant outcomes — a clear improvement from the earlier **53.8%**.


####  Confusion Matrix

[[11  6]
[ 9 13]]

| Term | Meaning | Count | Explanation |
|------|----------|--------|-------------|
| True Negatives (0,0) | Correctly predicted "No milestone" | 11 | The model correctly caught 11 plants that didn’t reach their milestone. |
| False Positives (0,1) | Wrongly predicted "Milestone achieved" | 6 | It thought 6 plants reached their milestone when they didn’t. |
| False Negatives (1,0) | Missed actual milestone achievers | 9 | It failed to identify 9 plants that actually reached it. |
| True Positives (1,1) | Correctly predicted "Milestone achieved" | 13 | It correctly identified 13 plants that truly reached their milestone. |

 Compared to before, the number of correct predictions (both 0s and 1s) has increased.  
 The model still occasionally confuses the two classes, but it’s improving.

---

#### Classification Report (Detailed Metrics)

| Class | Precision | Recall | F1-score | Interpretation |
|--------|------------|--------|-----------|----------------|
| **0 (No milestone)** | 0.55 | 0.65 | 0.59 | When predicting “No milestone”, it’s right about 55% of the time and captures 65% of all real non-milestones. |
| **1 (Milestone achieved)** | 0.68 | 0.59 | 0.63 | When predicting “Milestone achieved”, it’s correct 68% of the time and detects 59% of all true milestones. |

Overall performance metrics:
- **Accuracy:** 61.5%
- **Macro Average:** 0.62 (average performance across both classes)
- **Weighted Average:** 0.62 (accounts for class imbalance)

---

###  What Changed
- **Before tuning (C=1):** Accuracy ≈ 0.54  
- **After tuning (C=0.1):** Accuracy ≈ 0.62 ✅  
- This shows the model benefits from a little more regularization (simpler decision boundary).

---

###  Insights
- Logistic Regression can now generalize better — it’s not overfitting like before.
- It still struggles to perfectly separate the two outcomes (growth vs no growth), likely because:
  - The dataset might not have enough strong predictive patterns.
  - Some relationships may be nonlinear (not well captured by logistic regression).

---

### Next Steps
1. Try a **nonlinear model** like **Random Forest** or **Support Vector Machine (SVM)** to capture more complex patterns.  
2. Perform **feature engineering** — maybe create new features (like Temperature × Humidity) to capture interactions.  
3. Evaluate with **cross-validation** for a more reliable accuracy estimate.

---

**Summary:**  
> By scaling features and tuning the regularization parameter `C`, the model improved from 53.8% to 61.5% accuracy.  
> This shows that even small adjustments in preprocessing and model parameters can lead to better predictive performance.

In [12]:
# save the model
import joblib

joblib.dump(best_model, 'model.pkl')



['model.pkl']